# Convert .traj to webtraj

In [1]:
# write_webtraj(selection, io)

from json import dump
from array import array
from base64 import b64encode
from nanover.mdanalysis.converter import ELEMENT_INDEX

def base64(format, values):
    return b64encode(array(format, values)).decode("UTF-8")

def write_webtraj(selection, io):
    universe = selection.universe
    
    topology = {
        "elements": base64("B", (ELEMENT_INDEX[e] for e in selection.elements)),
        "bonds": base64("L", selection.bonds.to_indices().flat),
    }
    
    positions = []
    for t in universe.trajectory:
        positions.append(base64("f", (c * .1 for c in selection.positions.flat)))
    
    data = {
        "topology": topology,
        "positions": positions,
    }

    dump(data, io)

In [2]:
from nanover.mdanalysis.universe import universe_from_recording

universe = universe_from_recording(traj="gluhut_frame_int_1_slower_traj_4ns_4block.traj--SPLIT--1.traj")
selection = universe.select_atoms("not resname HOH")
universe, selection

(<Universe with 23070 atoms>, <AtomGroup with 183 atoms>)

In [ ]:
%%time
with open(f"webtraj.json", "w") as io:
    write_webtraj(selection, io)

CPU times: total: 16.3 s
Wall time: 16.8 s
